In [ ]:
# lsh init
lsh = MinHashLSH(threshold = 0.5, num_perm=128)
lsh.insert(blablabla)
# list of query
# to define
queries

#clustering

# list of list
clusters = []
# list of used fam
used_fams = []

def get_result(query, raw_result):
    filtered_results = [query] + [x for x in raw_result if x.split('-')[1] == query.split('-')[1]]
    return filtered_results

for query in queries:
    if query not in used_queries:        
        result = get_result(query, lsh.query(query))
        # add the new cluster in the list
        clusters.append(filtered_result)
        # exlude the results from the fams
        for f in filtered_result:
            if f not in used_queries:
                used_queries.append(f)
    

    


In [47]:
# from cluster, we have a list of fam. lets say famList[1,3,4,23,1234]
# recreate the matrix out of it. 
# n_hogs_in_cluster X n_species

cluster = [] # list of cluster previously done

import h5sparse

h5matrix = h5sparse.File(config.omadir + 'matrix.h5', mode='r')

arr = h5matrix['datasetName'][cluster[4]].toarray()

from sklearn import Covariance

cov = covariance().fit(arr)

model = covariance.GraphLassoCV()
model.fit(arr)
cov_ = model.covariance_


h5matrix.close()